In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np 
import math
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.callbacks import LambdaCallback


filename = 'Desktop\AAPL (3).csv'

df = pd.read_csv(filename)

df.sort_values('Date')

df.tail(10)

Using TensorFlow backend.


,Date,Open,High,Low,Close,Adj Close,Volume
4517,2018-08-31,226.509995,228.869995,226.000000,227.630005,227.630005,43340100
4518,2018-09-04,228.410004,229.179993,226.630005,228.360001,228.360001,27390100
4519,2018-09-05,228.990005,229.669998,225.100006,226.869995,226.869995,33333000
4520,2018-09-06,226.229996,227.350006,221.300003,223.100006,223.100006,34290000
4521,2018-09-07,221.850006,225.369995,220.710007,221.300003,221.300003,37619800
4522,2018-09-10,220.949997,221.850006,216.470001,218.330002,218.330002,39516500
4523,2018-09-11,218.009995,224.300003,216.559998,223.850006,223.850006,35749000
4524,2018-09-12,224.940002,225.000000,219.839996,221.070007,221.070007,49278700
4525,2018-09-13,223.520004,228.350006,222.570007,226.410004,226.410004,41706400
4526,2018-09-14,225.750000,226.839996,222.520004,223.839996,223.839996,31902700


In [2]:
high_prices = df.loc[:,'High'].as_matrix()
low_prices = df.loc[:,'Low'].as_matrix()
open_prices = df.loc[:,'Open'].as_matrix()
close_prices = df.loc[:,'Close'].as_matrix()
OHLC_mean = (high_prices+low_prices+open_prices+close_prices)/4.0

print(OHLC_mean)

[  4.13783425   4.263393     4.29380375 ... 222.71250125 225.21250525
 224.737499  ]


C:\Users\Yi Rong\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\Users\Yi Rong\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
C:\Users\Yi Rong\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Yi Rong\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [8]:
df['OHLC Mean'] = OHLC_mean
df.tail(10)

,Date,Open,High,Low,Close,Adj Close,Volume,OHLC Mean
4517,2018-08-31,226.509995,228.869995,226.000000,227.630005,227.630005,43340100,227.252499
4518,2018-09-04,228.410004,229.179993,226.630005,228.360001,228.360001,27390100,228.145001
4519,2018-09-05,228.990005,229.669998,225.100006,226.869995,226.869995,33333000,227.657501
4520,2018-09-06,226.229996,227.350006,221.300003,223.100006,223.100006,34290000,224.495003
4521,2018-09-07,221.850006,225.369995,220.710007,221.300003,221.300003,37619800,222.307503
4522,2018-09-10,220.949997,221.850006,216.470001,218.330002,218.330002,39516500,219.400002
4523,2018-09-11,218.009995,224.300003,216.559998,223.850006,223.850006,35749000,220.680001
4524,2018-09-12,224.940002,225.000000,219.839996,221.070007,221.070007,49278700,222.712501
4525,2018-09-13,223.520004,228.350006,222.570007,226.410004,226.410004,41706400,225.212505
4526,2018-09-14,225.750000,226.839996,222.520004,223.839996,223.839996,31902700,224.737499


In [3]:
scaler = MinMaxScaler()
OHLC_mean_scaled = OHLC_mean.reshape(-1,1)
OHLC_mean_scaled = scaler.fit_transform(OHLC_mean_scaled)
train_data = OHLC_mean_scaled[:4400]
test_data = OHLC_mean_scaled[4400:]

print(OHLC_mean_scaled)

[[0.01410179]
 [0.01465439]
 [0.01478824]
 ...
 [0.97609053]
 [0.98709353]
 [0.98500294]]


In [4]:
from numpy import newaxis
train_data = train_data.reshape(1,4400)
train_data = train_data[:,:,newaxis]
train_data.shape

(1, 4400, 1)

In [5]:
trainX = train_data[:,:-1,:]
trainY = train_data[:,1:,:]
print (trainX.shape)
print(trainY.shape)

(1, 4399, 1)
(1, 4399, 1)


In [6]:
model = Sequential()
model.add(LSTM(units=16, batch_input_shape=(1,None,1),  stateful = True, return_sequences=True))
model.add(LSTM(units=32, activation = 'relu', return_sequences = True))
model.add(Dense(units =1))
model.compile( loss='mean_squared_error', optimizer='adam')
resetCallback = LambdaCallback(on_epoch_begin=lambda epoch,logs: model.reset_states())
model.fit(trainX, trainY, batch_size=1, epochs=100, verbose = 1)

Epoch 1/100
1/1 [==============================] - 14s 14s/step - loss: 0.0839
Epoch 2/100
1/1 [==============================] - 10s 10s/step - loss: 0.0820
Epoch 3/100
1/1 [==============================] - 11s 11s/step - loss: 0.0797
Epoch 4/100
1/1 [==============================] - 11s 11s/step - loss: 0.0772
Epoch 5/100
1/1 [==============================] - 11s 11s/step - loss: 0.0747
Epoch 6/100
1/1 [==============================] - 11s 11s/step - loss: 0.0723
Epoch 7/100
1/1 [==============================] - 10s 10s/step - loss: 0.0699
Epoch 8/100
1/1 [==============================] - 12s 12s/step - loss: 0.0674
Epoch 9/100
1/1 [==============================] - 14s 14s/step - loss: 0.0648
Epoch 10/100
1/1 [==============================] - 12s 12s/step - loss: 0.0622
Epoch 11/100
1/1 [==============================] - 12s 12s/step - loss: 0.0595
Epoch 12/100
1/1 [==============================] - 11s 11s/step - loss: 0.0567
Epoch 13/100
1/1 [==============================]

In [9]:
#test_data = test_data.reshape(1, len(test_data),1)

model.reset_states()
trained_predictions = model.predict(train_data)
#tested_predictions = model.predict(test_data) #provided test data is given, but unlikely to be the case

print(trained_predictions[:,-1,:])

[[0.78009546]]


In [10]:
print(trained_predictions)
print (train_data)

[[[0.02881699]
  [0.03100983]
  [0.03284827]
  ...
  [0.77312225]
  [0.7769122 ]
  [0.78009546]]]
[[[0.01410179]
  [0.01465439]
  [0.01478824]
  ...
  [0.79393626]
  [0.78490283]
  [0.78310934]]]


In [11]:
model.reset_states()
predictions = model.predict(train_data)

futureElements = []

futureElement = predictions[:,-1:,:]
futureElements.append(futureElement)
print(futureElement)

for i in range(120):
    futureElement = model.predict(futureElement) #get the next step
    futureElements.append(futureElement) #store the future steps    

#after processing a sequence, reset the states for safety

model.reset_states()
print(futureElements)

[[[0.78009546]]]
[array([[[0.78009546]]], dtype=float32), array([[[0.1345452]]], dtype=float32), array([[[0.12007935]]], dtype=float32), array([[[0.10616377]]], dtype=float32), array([[[0.09457921]]], dtype=float32), array([[[0.08503747]]], dtype=float32), array([[[0.07724612]]], dtype=float32), array([[[0.07074952]]], dtype=float32), array([[[0.06535493]]], dtype=float32), array([[[0.06056781]]], dtype=float32), array([[[0.056368]]], dtype=float32), array([[[0.052972]]], dtype=float32), array([[[0.05023013]]], dtype=float32), array([[[0.0480097]]], dtype=float32), array([[[0.04611928]]], dtype=float32), array([[[0.04459485]]], dtype=float32), array([[[0.04336893]]], dtype=float32), array([[[0.04238619]]], dtype=float32), array([[[0.04160119]]], dtype=float32), array([[[0.04097669]]], dtype=float32), array([[[0.04048215]]], dtype=float32), array([[[0.04009257]]], dtype=float32), array([[[0.03978746]]], dtype=float32), array([[[0.0395501]]], dtype=float32), array([[[0.03936685]]], dtype